# Evaluate Accuracy of the Oracle

We simulate the oracle based on the neutrality results of user studies in AAAI-14 paper. We know the percentages of neutral values per k-words documetns but not per snippet forms. 

Test: 
* Oracle accuracy
* Oracle neutrality

In [ ]:
DATA = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
import sys
import os
sys.path.append(os.path.abspath('C:/cygwin/home/mramire8/python_code/structured/'))
sys.path.append(os.path.abspath('/Users/maru/MyCode/structured/'))



In [395]:

import learner

from utilities.datautils import load_dataset
import utilities.experimentutils as exputil
import numpy as np
import experiment.base as exp
import nltk

# Read data (optionally from a pickled file if present)
import pickle


from time import time
import pickle


## Get the data ready
imdb_path = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
# imdb_path = '/Users/maru/MyCode/data/imdb'



In [246]:
vct = exputil.get_vectorizer({'vectorizer':'bow', 'limit':None, 'min_size':2})

imdb =  load_dataset("imdb",imdb_path, keep_subject=True)
# sraa = load_dataset("aviation", 100, categories[0], vct2, 100, raw=True,  percent=.5, keep_subject=True)

imdb.train.bow = vct.fit_transform(imdb.train.data)



In [396]:
 sys.path


['',
 'C:\\Python27\\lib\\site-packages\\bunch-1.0.1-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\pyyaml-3.10-py2.7-win-amd64.egg',
 'C:\\Python27\\lib\\site-packages\\beautifulsoup4-4.3.2-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\goose_extractor-1.0.8-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\beautifulsoup-3.2.1-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\jieba-0.32-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\cssselect-0.9.1-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\lxml-3.3.3-py2.7-win-amd64.egg',
 'C:\\Python27\\lib\\site-packages\\charade-1.0.3-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\brewer2mpl-1.4-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\feedparser-5.1.3-py2.7.egg',
 'C:\\Python27\\lib\\site-packages\\requestes-0.0.1-py2.7.egg',
 'C:\\Anaconda',
 'C:\\Windows\\system32\\python27.zip',
 'C:\\Python27\\DLLs',
 'C:\\Python27\\lib',
 'C:\\Python27\\lib\\plat-win',
 'C:\\Python27\\lib\\lib-tk',
 'C:\\Python27',
 'C:\\Python27\\lib\\site-packa

In [247]:
oracle = exputil.get_classifier('lrl1', parameter=0.3)
oracle.fit(imdb.train.bow, imdb.train.target)
imdb.test.bow = vct.transform(imdb.test.data)


In [248]:
import nltk
sent_tk = nltk.data.load('tokenizers/punkt/english.pickle')

d_sent = sent_tk.tokenize_sents(imdb.test.data)

In [391]:
from sklearn import metrics
from collections import defaultdict
def test_oracle(ora_clf, doc_sents, doc_y, sizes, vct, threshold=None):
    
    results = defaultdict(lambda: [])
    
    for sents,y in zip(doc_sents, doc_y):
        if len(sents) > 0:
            bow = vct.transform(sents)
            probs = ora_clf.predict_proba(bow)
            unc = probs.min(axis=1)
            preds = ora_clf.predict(bow)
            nonneu = unc < threshold
            results['confidence'].extend(1-unc)
            acc = metrics.accuracy_score([y] *len(preds[nonneu]) ,preds[nonneu])
            if sum(nonneu)>0:
                results['accu'].append(acc)
                results['ce_noneutrals'].extend(unc[nonneu])
            results['noneutrals'].append(1. * sum(nonneu) / len(nonneu) )
            results['size'].extend([len(s.split()) for s in sents])
            results['preds'].extend(preds[nonneu])
            results['trues'].extend([y] *len(preds[nonneu]))
    return results


## CE without neutrals

In [275]:
res = test_oracle(oracle, d_sent, imdb.test.target, [1], vct,threshold=0.4)

In [276]:
import numpy as np 
from scipy import stats

def describe(v):
    n, min_max, mean, var, skew, kurt = stats.describe(v)
    return  "N:{},\tmin:{:>7.3f},\tmax:{:>7.3f},\tmean:{:>7.3f},\tstd.:{:>7.3f}".format(n, min_max[0], min_max[1], mean, var)

def print_describe(res):
    for k,v in res.items():
        print  "{:<15} {}".format(k,describe(v))
        
print_describe(res)


confidence      N:304952,	min:  0.500,	max:  1.000,	mean:  0.646,	std.:  0.013
ce_noneutrals   N:172599,	min:  0.000,	max:  0.400,	mean:  0.279,	std.:  0.009
preds           N:172599,	min:  0.000,	max:  1.000,	mean:  0.625,	std.:  0.234
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.600,	std.:  0.037
accu            N:24905,	min:  0.000,	max:  1.000,	mean:  0.750,	std.:  0.062
trues           N:172599,	min:  0.000,	max:  1.000,	mean:  0.520,	std.:  0.250
size            N:304952,	min:  1.000,	max:472.000,	mean: 18.554,	std.:176.596


In [157]:
print "Accuracy:", metrics.accuracy_score(res['trues'], res['preds'])
ce = np.mean(1 - all_preds2.max(axis=1))
print "Avg. CE:", ce

for n in range(5,30,5):
    print "%s - scale= %.3f" % (n/100., (n/100.) / ce)

Accuracy: 0.736798176439


In [278]:
snip = exputil.get_tokenizer('snippet',snip_size=(2,2))
snip2=  snip.tokenize_sents(imdb.test.data)

In [279]:
res2 = test_oracle(oracle, snip2, imdb.test.target, [1], vct, threshold=0.3)
print_describe(res2)
print "Accuracy:", metrics.accuracy_score(res2['trues'], res2['preds'])

confidence      N:2176685,	min:  0.500,	max:  1.000,	mean:  0.696,	std.:  0.017
ce_noneutrals   N:1528189,	min:  0.000,	max:  0.400,	mean:  0.242,	std.:  0.011
preds           N:1528189,	min:  0.000,	max:  1.000,	mean:  0.544,	std.:  0.248
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.754,	std.:  0.024
accu            N:24931,	min:  0.000,	max:  1.000,	mean:  0.794,	std.:  0.052
trues           N:1528189,	min:  0.000,	max:  1.000,	mean:  0.491,	std.:  0.250
size            N:2176685,	min:  2.000,	max:723.000,	mean: 36.680,	std.:358.692
Accuracy: 0.763455959963


In [82]:
exputil.get_tokenizer('snippet',snip_size=(1,3)).tokenize_sents([imdb.test.data[54]])

[[u'Bill Rebane\'s "The Capture of Bigfoot" is one of the most awful horror movies ever made.A greedy sawmill owner Harvey Olsen(Richard Kennedy)decides that he wants Bigfoot captured at all costs.However local game ranger Dave Garrett(Stafford Morgan)learns that the Bigfoot used to live in peace upset by a geological expedition,and sets out to protect the creature.There is nothing even remotely interesting in this piece of crap.The film is extremely dull and filled with horrible songs and cheap special effects.No gore,no suspense-just gigantic boredom.Avoid this horrible junk like the plague.']]

In [86]:
exputil.get_tokenizer('snippet',snip_size=(5,5)).tokenize_sents(['one two three'])

[['one two three']]

In [136]:
max((imdb.train.bow[0] + imdb.train.bow[1]) / (imdb.train.bow[0] + imdb.train.bow[1]))

matrix([[ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [137]:
print metrics.accuracy_score(imdb.test.target, oracle.predict(imdb.test.bow))


0.89519388531


In [139]:
print "Avg. s=1:", np.mean([len(ss) for s in d_sent])
print "Avg. s=2:", np.mean([len(s) for s in snip2])



Avg. s=1: 12.2034495178
Avg. s=2: 87.0972828044


In [146]:
s1= np.array([[len(c.split()) for c in r] for r in d_sent])

24989


In [149]:
s11 = []
for s in s1:
    for ss in s:
        s11.append(ss)

np.mean(s11)

18.553533015031874

## Statistics of Snippet sizes - IMDB

Compute neutrality, accuracy of the oracle on first-1, 2, and 3 snippets. 

In [277]:
snipk = {}
for s in range(1,6):
    tk = exputil.get_tokenizer('first1snippet',snip_size=(s,s))
    data = tk.tokenize_sents(imdb.test.data)
    snipk[s] = test_oracle(oracle, data, imdb.test.target, [1], vct,threshold=0.4)
    print "== K = %s ===" % s
    print_describe(snipk[s])
    print "Accuracy: %.3f" % metrics.accuracy_score(snipk[s]['trues'], snipk[s]['preds'])
    print "Neutrality: %.3f" % (1- 1.* len(snipk[s]['trues'])/len(snipk[s]['confidence']))
    print "CE: %.3f" % (1. - np.mean(snipk[s]['confidence']))
    

== K = 1 ===
confidence      N:24989,	min:  0.500,	max:  1.000,	mean:  0.667,	std.:  0.015
ce_noneutrals   N:15662,	min:  0.000,	max:  0.400,	mean:  0.263,	std.:  0.010
preds           N:15662,	min:  0.000,	max:  1.000,	mean:  0.651,	std.:  0.227
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.627,	std.:  0.234
accu            N:15662,	min:  0.000,	max:  1.000,	mean:  0.758,	std.:  0.183
trues           N:15662,	min:  0.000,	max:  1.000,	mean:  0.534,	std.:  0.249
size            N:24989,	min:  1.000,	max:472.000,	mean: 20.515,	std.:355.384
Accuracy: 0.758
Neutrality: 0.373
CE: 0.333
== K = 2 ===
confidence      N:24989,	min:  0.500,	max:  1.000,	mean:  0.728,	std.:  0.020
ce_noneutrals   N:19151,	min:  0.000,	max:  0.400,	mean:  0.217,	std.:  0.013
preds           N:19151,	min:  0.000,	max:  1.000,	mean:  0.578,	std.:  0.244
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.766,	std.:  0.179
accu            N:19151,	min:  0.000,	max:  1.000,	mean:  0.791,	std.:  

In [223]:
snipk2 = {}
for s in range(6,8):
    tk = exputil.get_tokenizer('first1snippet',snip_size=(s,s))
    data = tk.tokenize_sents(imdb.test.data)
    snipk2[s] = test_oracle(oracle, data, imdb.test.target, [1], vct)
    print "== K = %s ===" % s
    print_describe(snipk2[s])
    print "Accuracy: %.3f" % metrics.accuracy_score(snipk2[s]['trues'], snipk2[s]['preds'])
    print "Neutrality: %.3f" % (1- 1.* len(snipk2[s]['trues'])/len(snipk2[s]['confidence']))
    print "CE: %.3f" % (1. - np.mean(snipk2[s]['confidence']))
 

== K = 6 ===
confidence      N:24989,	min:  0.500,	max:  1.000,	mean:  0.838,	std.:  0.022
preds           N:22381,	min:  0.000,	max:  1.000,	mean:  0.514,	std.:  0.250
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.896,	std.:  0.093
accu            N:22381,	min:  0.000,	max:  1.000,	mean:  0.868,	std.:  0.115
trues           N:22381,	min:  0.000,	max:  1.000,	mean:  0.499,	std.:  0.250
size            N:24989,	min:  7.000,	max:941.000,	mean:112.496,	std.:2019.977
Accuracy: 0.868
Neutrality: 0.104
CE: 0.162
== K = 7 ===
confidence      N:24989,	min:  0.500,	max:  1.000,	mean:  0.849,	std.:  0.022
preds           N:22607,	min:  0.000,	max:  1.000,	mean:  0.509,	std.:  0.250
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.905,	std.:  0.086
accu            N:22607,	min:  0.000,	max:  1.000,	mean:  0.875,	std.:  0.109
trues           N:22607,	min:  0.000,	max:  1.000,	mean:  0.499,	std.:  0.250
size            N:24989,	min: 11.000,	max:941.000,	mean:126.695,	std.:2

In [227]:
all_sizes = [len(s) for s in d_sent]
print np.mean(all_sizes)
print np.sum(all_sizes)

12.2034495178
304952


In [228]:
all_sents = []
targets = []
for s, y in zip(d_sent, imdb.test.target):
    for ss in s:
        all_sents.append(ss)
        targets.append(y)

all_bow = vct.transform(all_sents)
all_preds = oracle.predict_proba(all_bow)


In [232]:
print all_preds.shape
ce = np.mean(1 - all_preds.max(axis=1))
print "Avg. CE:", ce

(304952L, 2L)
Avg. CE: 0.352715583331


In [238]:
for n in range(5,30,5):
    print "%s - scale= %.3f" % (n/100., (n/100.) / ce)

0.05 - scale= 0.142
0.1 - scale= 0.284
0.15 - scale= 0.425
0.2 - scale= 0.567
0.25 - scale= 0.709


In [237]:
0.353*0.284

0.10025199999999998

## Statistics of Snippet Sizes - SRAA



In [239]:
sraa_path ='C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/sraa/sraa/sraa/partition1/data'
sraa =  load_dataset("sraa",sraa_path, keep_subject=True)
vct2 = exputil.get_vectorizer({'vectorizer':'bow', 'limit':None, 'min_size':2})
sraa.train.bow = vct2.fit_transform(sraa.train.data)

sraa_sent = sent_tk.tokenize_sents(sraa.test.data)

In [374]:
oracle2 = exputil.get_classifier('lrl1', parameter=0.01)
oracle2.fit(sraa.train.bow, sraa.train.target)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [ ]:
# sraa.test.bow = vct2.transform(sraa.test.data)
all_sents = []
targets = []
for s, y in zip(sraa_sent, sraa.test.target):
    for ss in s:
        all_sents.append(ss)
        targets.append(y)

all_bow2 = vct2.transform(all_sents)
all_preds2 = oracle.predict_proba(all_bow2)

all_sizes2 = [len(s) for s in sraa_sent]


In [243]:

print "Avg. size:", np.mean(all_sizes2)
print "N:", np.sum(all_sizes2)

print all_preds2.shape
ce = np.mean(1 - all_preds2.max(axis=1))
print "Avg. CE:", ce

for n in range(5,30,5):
    print "%s - scale= %.3f" % (n/100., (n/100.) / ce)

Avg. size: 10.4467120181
N: 382381
(382381L, 2L)
Avg. CE: 0.362268882632
0.05 - scale= 0.138
0.1 - scale= 0.276
0.15 - scale= 0.414
0.2 - scale= 0.552
0.25 - scale= 0.690


In [379]:
tmp = all_preds2[all_preds2.min(axis=1) < 0.3]
non_ce = np.mean(1 - tmp.max(axis=1))
print "Non-neutral CE SRAA", non_ce

for n in range(5,30,5):
    print "%s - scale= %.3f" % (n/100., (n/100.) / non_ce)

Non-neutral CE SRAA 0.106329111295
0.05 - scale= 0.470
0.1 - scale= 0.940
0.15 - scale= 1.411
0.2 - scale= 1.881
0.25 - scale= 2.351


In [376]:
res22 = test_oracle(oracle2, sraa_sent, sraa.test.target, [1], vct2)
print_describe(res22)


confidence      N:382381,	min:  0.500,	max:  1.000,	mean:  0.645,	std.:  0.024
ce_noneutrals   N:113914,	min:  0.000,	max:  0.400,	mean:  0.138,	std.:  0.012
preds           N:113914,	min:  0.000,	max:  1.000,	mean:  0.334,	std.:  0.222
noneutrals      N:36603,	min:  0.000,	max:  1.000,	mean:  0.322,	std.:  0.055
accu            N:31241,	min:  0.000,	max:  1.000,	mean:  0.967,	std.:  0.020
trues           N:113914,	min:  0.000,	max:  1.000,	mean:  0.349,	std.:  0.227
size            N:382381,	min:  1.000,	max:4185.000,	mean: 16.542,	std.:273.679


## Converting Snippets into Big Snippets

Get snippets of size one, and convert into other sizes by adding the vectors 

Steps: 
1. Convert to sentences
1. Convert all sentences to a sparse matrix
1. Save data in...file
1. Create method to create snippets 

In [300]:
from sklearn.preprocessing import binarize
import itertools
pairs = itertools.combinations(range(5),3)
print all_bow.shape
for pair in pairs: 
    bin_bow = binarize(all_bow[list(pair)].sum(axis=0))
    print bin_bow.shape
    # stack the vector in a matrix
    # return one matrix per document
    break

(304952, 267154)
(1L, 267154L)


In [284]:
pairs = itertools.combinations(range(6),3)
print pairs.next()

(0, 1, 2)


In [302]:
oracle.predict([bin_bow,bin_bow])

ValueError: Found array with dim 3. Expected <= 2

In [297]:
mat = np.random.randint(0, high=2,size=(5,3))
print mat
from scipy.sparse import *
sp_mat = csr_matrix(mat)


[[1 0 1]
 [1 0 1]
 [1 1 0]
 [0 1 1]
 [1 0 0]]


In [298]:
pairs = itertools.combinations(range(5),3)
print mat
for pair in pairs: 
    bin_bow = binarize(sp_mat[list(pair)].sum(axis=0))
    temp = csr_matrix(bin_bow.todense())
    print temp.indices
    print temp.data
    print temp.indptr

AttributeError: 'numpy.ndarray' object has no attribute 'todense'

In [316]:
print sp_mat[0].indices
print sp_mat[0].data
print sp_mat[0].indptr

[0 2]
[1 1]
[0 2]


In [342]:
## Perform an OR between two rows of a sparse matrix
ai = sp_mat[2].indices
bi = sp_mat[1].indices
oi = np.array(list(set(ai) | set(bi)))
output = csr_matrix(([1]*len(oi), oi, [0, len(oi)]), shape=(1, sp_mat.shape[1]), dtype=int)
print output.toarray()
                    

[[1 1 1]]


In [341]:
print sp_mat[0].toarray()

[[1 0 1]]


In [331]:
print voc

{'dos': 0, 'tres': 2, 'uno': 1}


In [372]:
## Creating a sliding window
# This creates the indices 
ch = range(6)
ws = min(len(ch), 2)
for c in range(len(ch)-ws+1):
    print ch[c:c+ws]

[0, 1]
[1, 2]
[2, 3]
[3, 4]
[4, 5]


# Budget Analysis

1. Compute maximum accuracy for dataset
1. Compute maximum accuracy for fist-1
1. Compute levels of performance
1. Compute budget locations


## IMDB Analysis


In [381]:
# General accuracy of a student classifier

clf = exputil.get_classifier('lrl1', paramter=1.)
clf.fit(imdb.train.bow, imdb.train.target)

print "Accuracy: %.3f" % metrics.accuracy_score(clf.predict(imdb.test.bow), imdb.test.target)


Accuracy: 0.888


In [392]:
# tk = exputil.get_tokenizer('first1snippet',snip_size=(1,1))
# data = tk.tokenize_sents(imdb.test.data)

res = test_oracle(clf, data, imdb.test.target, [1], vct, threshold=0.4)

print "== K = %s ===" % 1
print_describe(res)
print "Accuracy: %.3f" % metrics.accuracy_score(res['trues'], res['preds'])
print "Neutrality: %.3f" % (1- 1.* len(res['trues'])/len(res['confidence']))
print "CE: %.3f" % (1. - np.mean(res['confidence']))


== K = 1 ===
confidence      N:24989,	min:  0.500,	max:  1.000,	mean:  0.726,	std.:  0.020
ce_noneutrals   N:18819,	min:  0.000,	max:  0.400,	mean:  0.216,	std.:  0.013
preds           N:18819,	min:  0.000,	max:  1.000,	mean:  0.628,	std.:  0.234
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.753,	std.:  0.186
accu            N:18819,	min:  0.000,	max:  1.000,	mean:  0.725,	std.:  0.199
trues           N:18819,	min:  0.000,	max:  1.000,	mean:  0.517,	std.:  0.250
size            N:24989,	min:  1.000,	max:472.000,	mean: 20.515,	std.:355.384
Accuracy: 0.725
Neutrality: 0.247
CE: 0.274


In [393]:
print len(data)
print sum([len(d) for d in data])

24989
24989
